In [ ]:
import time
from pprint import pprint, pformat
from itertools import product

In [ ]:
#!pip install Cython
%load_ext cython

In [ ]:
%%cython

cdef int pos
cdef bytes subject

cdef void factor():
    global pos, subject
    if   subject[pos] == ord('+'): pos += 1; factor()
    elif subject[pos] == ord('-'): pos += 1; factor()
    elif subject[pos] == ord('('):
        pos += 1
        expr()
        if subject[pos] == ord(')'): pos += 1
        else: raise Exception(pos)
    elif subject[pos] == ord('x'): pos += 1
    elif subject[pos] == ord('y'): pos += 1
    elif subject[pos] == ord('0'): pos += 1
    elif subject[pos] == ord('1'): pos += 1
    else: raise Exception(pos)

cdef void term():
    global pos, subject
    factor()
    while subject[pos] == ord('*') or subject[pos] == ord('/'):
        pos += 1
        term()

cdef void expr():
    global pos, subject
    term()
    while subject[pos] == ord('+') or subject[pos] == ord('-'):
        pos += 1
        expr()

cdef void stmt():
    global pos, subject
    expr()
    if subject[pos] != ord('\n'):
        raise Exception(pos)

def parse_expression(s):
    global pos; pos = 0
    global subject; subject = f"{s}\n".encode('ascii')
    try: stmt()
    except Exception as e:
      return e.args[0]+1
    return 0

In [ ]:
total = None
tokens = "(x*-1+0/y)"
def expressions(length):
    global total; total = 1
    for toks in product(tokens, repeat=length):
        expression = "".join(toks)
        yield (parse_expression(expression), expression)
        total += 1
    total -= 1

In [ ]:
for length in range(1, 8):
    start = time.time_ns() // 1000
    classes = [0] * (length+2)
    for expr in expressions(length):
        classes[expr[0]] += 1
    finish = time.time_ns() // 1000
    classes = [(c * 100.0 / total) for c in classes]
    pprint([length, total, (finish - start) / 1000000.0, classes])

In [ ]:
token_map = {'(': 1, 'x': 2, '0': 3, '+': 4, '*': 5, '/': 6, '-': 7, '1': 8, 'y': 9, ')': 10}
def EXPRESSIONS(length):
    XS = []
    for expr in expressions(length):
        XS.append([expr[0]] + [token_map[x] for x in expr[1]])
    return XS

In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

results = {}
for length in range(1, 6):
    columns = ['target'] + [f"x{i}" for i in range(1, length+1)]
    data = pd.DataFrame(EXPRESSIONS(length), columns=columns)
    X = data.iloc[:, 1:length+1]
    y = data.iloc[:, 0]
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=53, stratify=y
    )
    classifiers = {
        "Logistic Regression": LogisticRegression(max_iter=200),
        "Support Vector Classifier": SVC(),
        "Random Forest": RandomForestClassifier(n_estimators=100)
    }
    results[length] = {}
    for name, model in classifiers.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        results[length][name] = {
            "Accuracy": accuracy_score(y_test, y_pred),
            "Precision": precision_score(y_test, y_pred, average='weighted'),
            "Recall": recall_score(y_test, y_pred, average='weighted'),
            "F1 Score": f1_score(y_test, y_pred, average='weighted'),
            "Classification Report": classification_report(y_test, y_pred)
        }
    comparison = pd.DataFrame(
    { model: {
          "Accuracy": metrics["Accuracy"],
          "Precision": metrics["Precision"],
          "Recall": metrics["Recall"],
          "F1 Score": metrics["F1 Score"]
      } for model, metrics in results[length].items()
    }).T

    print("\n")
    print(comparison)
    print("\n")

    if False:
        for model, metrics in reports[length].items():
            print(f"Classification Report: {model}:")
            print(metrics["Classification Report"])

In [ ]:
!pip install torch torchvision torchaudio

In [ ]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.model_selection import train_test_split
# os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

for length in range(1, 7):
    data = np.array(EXPRESSIONS(length))
    X = data[:, 1:].astype(np.int64) # tokens (shape: [n_samples, 5])
    y = data[:, 0].astype(np.int64) # target column
    num_classes = int(y.max()) + 1
    seq_length = X.shape[1] # should be 5 as specified
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=53)
    batch_size = 32
    train_dataset = TensorDataset(
        torch.tensor(X_train, dtype=torch.long),
        torch.tensor(y_train, dtype=torch.long))
    test_dataset = TensorDataset(
        torch.tensor(X_test, dtype=torch.long),
        torch.tensor(y_test, dtype=torch.long))
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    vocab_size = 11 # tokens 1-10
    embedding_dim = 8 # arbitrary embedding dimension
    hidden_dim = 32 # used by hidden layers / RNN
    LEARNING_RATE = 0.001
    EPOCHS = 20

    # Model 1: Logistic Regression (a single linear layer after embedding+flattening)
    class LogisticRegressionModel(nn.Module):
        def __init__(self, vocab_size, embedding_dim, num_classes, seq_length):
            super().__init__()
            self.embedding = nn.Embedding(vocab_size, embedding_dim)
            self.fc = nn.Linear(seq_length * embedding_dim, num_classes)

        def forward(self, x):
            # x: [batch_size, seq_length]
            x = self.embedding(x) # shape: [batch_size, seq_length, embedding_dim]
            x = x.view(x.size(0), -1) # flatten to [batch_size, seq_length*embedding_dim]
            out = self.fc(x)
            return out

    # Model 2: Feedforward network with one hidden layer
    class FeedforwardNN(nn.Module):
        def __init__(self, vocab_size, embedding_dim, num_classes, seq_length, hidden_dim):
            super().__init__()
            self.embedding = nn.Embedding(vocab_size, embedding_dim)
            self.fc1 = nn.Linear(seq_length * embedding_dim, hidden_dim)
            self.relu = nn.ReLU()
            self.fc2 = nn.Linear(hidden_dim, num_classes)

        def forward(self, x):
            x = self.embedding(x)
            x = x.view(x.size(0), -1)
            x = self.relu(self.fc1(x))
            out = self.fc2(x)
            return out

    # Model 3: Deeper network with two hidden layers
    class DeeperNN(nn.Module):
        def __init__(self, vocab_size, embedding_dim, num_classes, seq_length, hidden_dim):
            super().__init__()
            self.embedding = nn.Embedding(vocab_size, embedding_dim)
            self.fc1 = nn.Linear(seq_length * embedding_dim, hidden_dim)
            self.fc2 = nn.Linear(hidden_dim, hidden_dim)
            self.fc3 = nn.Linear(hidden_dim, num_classes)
            self.relu = nn.ReLU()

        def forward(self, x):
            x = self.embedding(x)
            x = x.view(x.size(0), -1)
            x = self.relu(self.fc1(x))
            x = self.relu(self.fc2(x))
            out = self.fc3(x)
            return out

    # Model 4: RNN using an LSTM (the hidden state of the last time step is used for classification)
    class RNNModel(nn.Module):
        def __init__(self, vocab_size, embedding_dim, num_classes, hidden_dim, num_layers=1):
            super().__init__()
            self.embedding = nn.Embedding(vocab_size, embedding_dim)
            self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers, batch_first=True)
            self.fc = nn.Linear(hidden_dim, num_classes)

        def forward(self, x):
            x = self.embedding(x) # [batch, seq_length, embedding_dim]
            out, (hn, cn) = self.lstm(x) # hn: [num_layers, batch, hidden_dim]
            out = self.fc(hn[-1]) # use last layer’s hidden state for classification
            return out

    # Model 5: Transformer-based Model (using a single encoder layer)
    class TransformerModel(nn.Module):
        def __init__(self, vocab_size, embedding_dim, num_classes, seq_length, nhead=2, num_encoder_layers=1, dim_feedforward=64):
            super().__init__()
            self.embedding = nn.Embedding(vocab_size, embedding_dim)
            # Learnable positional encoding (for simplicity)
            self.pos_embedding = nn.Parameter(torch.randn(1, seq_length, embedding_dim))
            encoder_layer = nn.TransformerEncoderLayer(d_model=embedding_dim, nhead=nhead, dim_feedforward=dim_feedforward)
            self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_encoder_layers)
            self.fc = nn.Linear(seq_length * embedding_dim, num_classes)

        def forward(self, x):
            x = self.embedding(x) + self.pos_embedding # [batch, seq_length, embedding_dim]
            x = x.transpose(0, 1) # Transformer expects: [seq_length, batch, embedding_dim]
            x = self.transformer_encoder(x)
            x = x.transpose(0, 1) # back to [batch, seq_length, embedding_dim]
            x = x.reshape(x.size(0), -1) # flatten
            out = self.fc(x)
            return out

    # Model 6: Convolutional Neural Network (CNN) with 1D Convolutions
    class CNNModel(nn.Module):
        def __init__(self, vocab_size, embedding_dim, num_classes, seq_length, num_filters=16, kernel_size=2):
            super().__init__()
            self.embedding = nn.Embedding(vocab_size, embedding_dim)
            # For conv1d, we expect input of shape (batch, channels, seq_length)
            self.conv1 = nn.Conv1d(in_channels=embedding_dim, out_channels=num_filters, kernel_size=kernel_size)
            conv_output_size = seq_length - kernel_size + 1
            self.fc = nn.Linear(num_filters * conv_output_size, num_classes)
            self.relu = nn.ReLU()

        def forward(self, x):
            x = self.embedding(x) # [batch, seq_length, embedding_dim]
            x = x.transpose(1, 2) # [batch, embedding_dim, seq_length]
            x = self.conv1(x) # [batch, num_filters, new_seq_length]
            x = self.relu(x)
            x = x.view(x.size(0), -1) # flatten
            out = self.fc(x)
            return out

    def train_model(model, train_loader, criterion, optimizer, device, epochs=20):
        model.train()
        for epoch in range(epochs):
            total_loss = 0.0
            for batch_x, batch_y in train_loader:
                batch_x = batch_x.to(device)
                batch_y = batch_y.to(device)
                optimizer.zero_grad()
                outputs = model(batch_x)
                loss = criterion(outputs, batch_y)
                loss.backward()
                optimizer.step()
                total_loss += loss.item()
            avg_loss = total_loss / len(train_loader)
            print(f"Epoch [{epoch+1}/{epochs}], Loss: {avg_loss:.4f}")

    def evaluate_model(model, test_loader, device):
        model.eval()
        all_preds = []
        all_labels = []
        with torch.no_grad():
            for batch_x, batch_y in test_loader:
                batch_x = batch_x.to(device)
                outputs = model(batch_x)
                _, preds = torch.max(outputs, 1)
                all_preds.append(preds.cpu())
                all_labels.append(batch_y)
        all_preds = torch.cat(all_preds).numpy()
        all_labels = torch.cat(all_labels).numpy()
        return all_preds, all_labels

    models = {
        "Logistic Regression": LogisticRegressionModel(vocab_size, embedding_dim, num_classes, seq_length),
        "Feedforward NN": FeedforwardNN(vocab_size, embedding_dim, num_classes, seq_length, hidden_dim),
        "Deeper NN": DeeperNN(vocab_size, embedding_dim, num_classes, seq_length, hidden_dim),
        "RNN (LSTM)": RNNModel(vocab_size, embedding_dim, num_classes, hidden_dim),
        "Transformer": TransformerModel(vocab_size, embedding_dim, num_classes, seq_length),
        "CNN": CNNModel(vocab_size, embedding_dim, num_classes, seq_length)
    }

    results_dict = {}
    for model_name, model in models.items():
        print(f"Training model: {model_name}")
        model.to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
        train_model(model, train_loader, criterion, optimizer, device, epochs=EPOCHS)
        preds, labels = evaluate_model(model, test_loader, device)
        acc = accuracy_score(labels, preds)
        prec = precision_score(labels, preds, average='weighted', zero_division=0)
        rec = recall_score(labels, preds, average='weighted', zero_division=0)
        f1 = f1_score(labels, preds, average='weighted', zero_division=0)
        report = classification_report(labels, preds, zero_division=0)
        results_dict[model_name] = {
            "Accuracy": acc,
            "Precision": prec,
            "Recall": rec,
            "F1 Score": f1,
            "Classification Report": report
        }
        print(f"Classification report for {model_name}:\n{report}")

    summary_data = []
    for name, metrics in results_dict.items():
        summary_data.append({
            "Model": name,
            "Accuracy": metrics["Accuracy"],
            "Precision": metrics["Precision"],
            "Recall": metrics["Recall"],
            "F1 Score": metrics["F1 Score"]
        })

    summary = pd.DataFrame(summary_data)
    print("Summary of Evaluation Metrics:")
    print(summary)